# Working with APIs in Python - Harvard Art Museum - Working Notebook

This is the Jupyter Notebook you will use for the rest of this unit. We'll be using it to retrieve information from the Harvard Art Museum API. In working with this notebook, you will learn how to construct and modify queries to get tailored results, as well as a bit about reading API documentation and understanding the data, once you've got it. 

To get started, check out the Museum's [webpage describing its API](https://www.harvardartmuseums.org/collections/api).

## Authentication

First, you'll need an <b>API key</b>. Every time you submit a query to the Museum's API, you'll need to  validate your request by including this key, which is unique to you. You can sign up for one through the [main page](https://www.harvardartmuseums.org/collections/api). Once you fill out the request form, you can expect to receive your key by e-mail in a few seconds. Once you have it, copy it into a document that's readily available. You may want to set up a cheat sheet that you can return to throughout the course for just this purpose.

Keep in mind that other APIs may require different kinds of authentication. Sometimes, it can get pretty complicated. If it does, look for documentation on that particular website about how to use it. Thankfully, the Art Museum's API authentication is relatively simple, which makes things easy for us.

Once you have your key, we'll make it a <b>global variable</b>, which you can use throughout this notebook. Let's call it "APIKEY".

In [ ]:
APIKEY=""

Good. Since we're just getting started, let's make sure that this variable was assigned correctly.

In [ ]:
APIKEY

## 1. Express a Request as a URL

When developers create <b>REST APIs</b>, they make it so that each request for data can be expressed as a specific <b>URL</b>. For the Art Museum, that URL needs four components: a base URL, an <b>endpoint</b>, a set of <b>query parameters</b>, and your personal API key. 

It will look like this:

```python
url = "https://api.harvardartmuseums.org/" + endpoint + "?" + query + "&apikey=" + APIKEY
```

The question mark acts as a separator between your endpoint URL and your query parameters. You typically use ampersands (`"&"`) to separate different query parameters. `APIKEY` is a required parameter to use the Art Museum API - if a valid key is not included, you'll receive a forbidden error.

Since you've already set your personal API key as a variable, let's go ahead and construct a simple URL that will retrieve information from the Art Museum's [Object endpoint](https://github.com/harvardartmuseums/api-docs/blob/master/sections/object.md). The URL will need to conform to the Museum's API documentation.

Let's look for paintings by the artist Jacques Louis David. We'll surround his name in quotes so it matches exactly. We'll also specify that we only want objects classified as paintings. Finally, we'll sort our results by pageviews, so we see the most viewed object records first. Once you've got your complete URL, go ahead and paste it into address bar for your web browser.

In [ ]:
base_url = "https://api.harvardartmuseums.org/"
endpoint = "object"
query ='person="Jacques Louis David"&classification=Paintings&sort=totalpageviews&sortorder=desc'
url_api_key = "&apikey=" + APIKEY

url_request = base_url + endpoint + "?" + query + url_api_key
print(url_request)

Congratulations! Having run this cell, you've now sent your first request to Harvard Art Museum's API.

In the video accompanying this notebook, Cole mentions that you should easily be able to find a browser extension so that the results are presented in a more friendly format. If you'd like to add one, try an internet search for "JSON browser extension," in addition to the name of the browser you're using.

We can also do a bit of string formatting so the string displays as a URL you can click. The string `format()` method allows you to interpolate variables or expressions sequentially into a string. You use curly braces (`{}`) in the string where you'd like to substitute variables; you can also use named arguments (`"Test {foo}".format(foo="bar")` prints "Test bar").

The `display` and `HTML` libraries allow us to display HTML formatted text directly in our notebook.

In [ ]:
from IPython.core.display import display, HTML
query_url = "<a href='{}'>{}</a>".format(url_request, url_request)
display(HTML(query_url))

## 2. Improve Requests with Python

The results you see are in a format called JSON, which we'll discuss in a moment. For now, it's worth pointing out that this isn't a terribly efficient way to structure a search, especially if you want to specify a lot of parameters. We're going to want to manipulate our results to extract information from it, and to do that, we'll need to try a different approach.

Instead, let's write a new request in Python and get the results directly in this notebook.

To do that, let's start by importing the Python [`requests` library](https://requests.readthedocs.io/en/master/). In coding, <b>libraries</b> are extremely helpful, so we encourage you to use them as often as possible. A library is a collection of code that's been written in advance, so you can repurpose existing code, focus on higher-order logic by abstracting lower-level tasks, and streamline what you have to write. The `requests` library makes submitting complex API queries much easier.

In [ ]:
import requests

In this example, we'll stick with the Object endpoint, but now let's look for records having to do with holdings from the 18th century. We can further narrow the search by looking for the ten objects that have been viewed the most on the Museum's website.

In [ ]:
url = "https://api.harvardartmuseums.org/object"
parameters = {
    "century": "18th century",
    "classification": "Paintings",
    "sort": "totalpageviews",
    "sortorder": "desc",
    "size": 10,
    "apikey":APIKEY
}
R = requests.get(url,params=parameters)
R.json()

<br />
The request syntax itself is so simple, you might have missed it. It's just a single line of Python that submits your request to the Art Museum's API.

In [ ]:
R = requests.get(url,params=parameters)

Objects retrieved using the `requests` library have a built-in method, `.json()`, which converts a JSON file received as a response to a request from a string of text that happens to be in this data format into Python-native data structures, like lists, dictionaries, numbers and strings. We can use this method to see a dictionary representation of what we've gotten from the API request.

In [ ]:
top_18c_paintings = R.json()
top_18c_paintings

Now that we have these ten results, we can view the images for the top hits.

In [ ]:
for painting in top_18c_paintings['records']:
    painting_image_url = "<a href='{}'>{}</a>".format(painting['primaryimageurl'], painting['primaryimageurl'])
    print(painting['title'])
    display(HTML(painting_image_url))

If you look at the top image, you will likely see an image of Madame Pompadour, one of the most powerful women of the period. (NB: You may see something different, of course, because the Art Museum's "toppageviews" data may have changed since we ran this cell.) If you go back to the query itself and replace `Paintings` with `Sculpture,` or `Drawings`, or `Prints`, or `Vessels` you'll see the ten most-viewed objects in that medium from the eighteenth century. 

You're starting to see the how APIs can help you sift through massive amounts of data. Before we go any further, let's pause for a moment to learn more about JSON, the data format in which your results appear.

### Proceed to the next video, in which Christine discusses JSON.


## A Closer Look at the Art Museum's JSON Response

Just as we saw in the last video, the output you get from Harvard Art Museum is structured as <b>JSON</b> (JavaScript Object Notation). [JSON is described](https://www.json.org/json-en.html) as "a lightweight data-interchange format. It is easy for humans to read and write. It is easy for machines to parse and generate." It can still seem pretty complicated!

Let's print our `top_18c_paintings` variable again:

In [ ]:
top_18c_paintings

Try searching (`Ctrl+F` in your browser window, rather than Jupyter, which will only search code/Markdown cells) to find the key `people`, for instance. In this case, we have an array that contains an object, which itself contains a list, the information for François Boucher, who painted Madame Pompadour.

Data can be retrieved from dictionaries easily, so you can get a value from its key very quickly, no matter how large the dictionary is. However, they are also unordered, so if you iterate through all of the key / value pairs in the dictionary, there's no guarantee that they'll be in the same order.

The `requests` library doesn't just return our JSON data; it has a number of functions and attributes which makes it possible for us to interact with the HTTP response. We saved the response as `R`:

In [ ]:
R

Printing `R` shows the HTTP response code for the request - in this case, `200`, which is a successful response. You can find out more about response codes [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status). Try submitting a request without an API key - you'll get a `401` unauthorized error code instead.

Here are some more attributes of the `Response`:

In [ ]:
print(type(R)) # Print the type of the variable R
print()
print(R.url) # Print our query URL
print()
print(R.headers)

We're mostly interested in the data we get back from the response. We could access this through `R.text` or `R.content`, which return the response body as a string and as bytes (useful when requesting media). However, as we've seen, `requests` also includes a handy JSON decoder method, which transforms our JSON response into a Python dictionary:

In [ ]:
print(type(R.text))
print(type(R.content))
print(type(R.json()))

In [ ]:
type(R.json())

The parameters that we submitted in our request are themselves in the format of a dictionary, and we can access each of its values through its keys.

## 3. Modifying your Request

Let's say we're not interested in the eighteenth century, but rather in obscure contemporary works of art. There are a few ways we might go about doing this. One way might be to sort our search results by `totalpageviews`, but this time in ascending order, so that we get the least viewed items first.

Let's make sure that the objects we retrieve have an image associated with them, as well. To do that, we can go back to the [API documentation](https://www.harvardartmuseums.org/collections/api) and look for hints about what we might be able to do.

Really dig into this cell and practice changing the parameters. We've given you some hints for ways that you might replace certain parameters or introduce new ones, so that you can tailor your results.

In [ ]:
url = "https://api.harvardartmuseums.org/object"
parameters = {
    "century":"20th century",
    "classification":"Photographs",
    "sort": "totalpageviews",
    "sortorder": "asc",
    "size":10,
    "apikey":APIKEY,
}

practice_R = requests.get(url,params=parameters)
practice_R.json()

# Contemporary Mixed Media from Across Asia:
#    "century":"21st century",
#    "place":"China|Japan|Korea|Mongolia|Burma|Cambodia|Indonesia|Malaysia|Philippines|Thailand|Vietnam",
#    "classification":"Performance Artifacts|Multiples|Jewelry|Audiovisual Works|Sculpture|Unidentified|Textile Arts|Portfolios|Photographs",

# A Number of Important 20C Artists
#    "person":"Matisse|Gogh|Picasso|Bonnard|Derain|Dali|Warhol|Kandinsky|Duchamp|Beckmann",

# 20th Century Sculptures
#    "century":"20th century",
#    "classification":"Sculpture",

# Madonna and Child 
#    "title":"Madonna|Virgin|Pieta|Pietà",

### Let's Get Some Cleaner Results

All this is well and good, but reading JSON can quickly become tedious. You'll often want to look at specific attributes instead of a big blob of data. Since the API returns everything, you'll have to format the output in a friendly, more readable format. In this next cell, we'll iterate through the results and print them in a more forgiving format.

We're being pretty low-level with the text formatting here, and one important key to understanding this bit is that "\t" means "tab", so that you can insert that character, which normally does something else.

Feel free to play around with this cell to format it more to your liking. Again, we've provided a bit of a hint to get you started. You might want to change `R`, the results from our 18th-century search, to `practice_R`, so that you can see the results from the previous cell.

In [ ]:
records = R.json()['records']
print("views\ttitle")
print()
for record in records:    
    print("{}\t{}".format(record['totalpageviews'],record['title']))
    print()

## Using a Function to Query Endpoints

You might notice, looking at the documentation, that we've only been accessing the `objects` API endpoint, when there are many others that we could query.

Endpoints on the same API are likely to behave similarly, but they will serve different purposes. Looking at our HAM endpoints, it looks like they all follow the same basic formulation: `https://api.harvardartmuseums.org/RESOURCE_TYPE`. We can use this to our advantage, and create a function to query any endpoint easily.

We're presuming that you have some familiarity with creating functions and using keyword arguments in Python. If this is new to you, there are lots of resources to read on the web, like [this easy tutorial](https://www.w3schools.com/python/python_functions.asp).

In the cell below, we're going to define a new function `ham_query`. The main part of our function is `requests.get(url,params=params)` the same query that we've been using throughout this notebook. Functions receive information through `arguments` (aka `parameters`). In this case, there are three, the `endpoint` string (so far we've been using `object`), an `apikey` (in this case our global variable `APIKEY`), and a series of unspecified keyword arguments, or `**kwargs`. (We'll get into this in a moment.)

In [ ]:
def ham_query(endpoint, apikey=APIKEY, **kwargs):
    """Sends kwargs to the specified endpoint, using apikey for authentication"""
    params = kwargs
    params['apikey'] = apikey
    url = "https://api.harvardartmuseums.org/{}".format(endpoint)
    R = requests.get(url,params=params)
    return R

Once this functon is defined, all we need is one simple line of code to query the API, and another to view the response in JSON format:

In [ ]:
response = ham_query("object", century="18th century")
response.json()

Each time we run or "call" this function, we need to send information for each of its three parameters, `endpoint`, `apikey`, and `**kwargs`. Since `apikey` is already defined, we just need to specify the other two.

Keyword arguments are a bit complicated. Essentially, we'll use them to load our desired parameters for each query. Regular arguments take strings, but as you can see above, `**kwargs` require a `key = value` syntax. When you write your function, neither the key nor the value need be specified. This is useful for us, because each endpoint in the HAM API makes use of different parameters. 

So, for instance, if we were using the `object` endpoint, we might specify `century=18th century`. If you look at the [API documentation](https://github.com/harvardartmuseums/api-docs), though, you'll see that there's no such parameter for the `gallery` endpoint, which contains information about the physical space fo the museum. Instead, we might use `floor=2` to see which galleries exist on the second floor.

In [ ]:
response = ham_query("gallery", floor=2)
response.json()

Now, go ahead and try out some other endpoints. Again, the [API documentation](https://github.com/harvardartmuseums/api-docs) is your friend, as it will tell you what parameters can be used with each endpoint. We'll get you started with this example, that prints current exhibits with their start and end dates:

In [ ]:
response = ham_query("exhibition", status="current")
current_exhibits = response.json()['records']
current_exhibits
print()
for exhibit in current_exhibits:
    print("{} ({} to {})".format(exhibit['title'],exhibit['begindate'],exhibit['enddate'])) 

## 4. Using APIs for Research: Finding Artworks by Eighteenth-Century Women

So far, we've only been getting limited sets of data. But what if there were a big query we wanted to make? Let's say, for instance, that we wanted to examine every work of art in Harvard's collection made by a woman in the eighteenth century. This isn't a problem we can solve using [the Museum's search engine](https://harvardartmuseums.org/collections), as there is no filter for sex or gender. To do this, we need to apply what we've learned about APIs. First, let's get a sense of the scale of the problem by doing a big request for all 18C objects in the collection, using our new query function.

Here, for the first time, we'll pass in more than one keyword argument, which `**kwargs` allows.

In [ ]:
records = ham_query("object", century="18th century", sort="totalpageviews", sortorder="desc", size=100, page=1).json()

Looking at our previous queries, it seems as though we've got some information about our query in the "info" section. Let's take a look at that.

(Keep in mind that the results that you see are likely to differ from what we found when we ran this query in the accompanying video. The Art Museum's holdings may have changed in the meantime.)

In [ ]:
records['info']

With 181 pages to go through, we'll certainly need to write a script to check for objects by women throughout the whole of the Museum's eighteenth-century holdings. Before we do, let's get a bit more information about our data. We'll be able to use `type` to check what sort of variable or Python object we have. The method `keys` will allow us to check the top-level keys in our dictionary. Finally, by just entering the name of our dictionary as a machine-readable object, we'll be able to view all of its content.

In [ ]:
records.keys()

In [ ]:
records

In [ ]:
records['records'][0]['people'][0]

### Finding Works by Women - Iterating through JSON

With almost 20,000 records to review, this is definitely a problem we're going to want to solve computationally. Before we continue, let's try to get a rough sense of the number of objects by women in the collection's eighteenth-century holdings as a whole. To do this, we'll have to write a loop to iterate over each record, looking for the female artists.

As we say using the `keys` method, it looks as though we have two components to our response, info and records. Since `info` is request specific, we're just after `records`, we'll want to isolate that as `extracted_records`.

Let's start by looking for men, as there are so many more of them.

In [ ]:
extracted_records = records["records"]
print("name\t\t\ttitle")
print()
for record in extracted_records:
    if record['people'][0]['gender'] == "male":
        print("{}\t{}".format(record['people'][0]['name'],record['title']))

That can't be right. In the video, when we ran this script, it produced list of works by male artists. But it seemed to have encountered a problem around the result for _The Beloved Child_, by Fragonard. <b>You'll likely see something a bit different, when you run this same script, because the Art Museum's data has likely changed somewhat.</b> The clue to what's wrong is the message at the bottom of the red field: `KeyError: 'people'`. Let's look a little more closely at this part of the data.

There's an obvious problem here with `people` field - a woman participated in its creation, but we aren't picking that up. But this doesn't seem to be what's causing the script to fail. The `people` key is intact. Try checking records nearby (`10`, `11`, `13`, and `14`), and, when you do, you can replace the `people` key with `title` and `primaryimageurl` to learn a bit more about these items.

In [ ]:
extracted_records[11]['people']

When we ran this notebook, it was clearly the 14th record that was causing the problem: ["Moon Flask with Decoration of the Eight Buddhist Treasures"](https://ids.lib.harvard.edu/ids/view/10866916). You'll likely find that it's ranked differently. This record lacks a `people` field, so we'll have to correct for that in the script, and only check those records that have the `people` key.

In solving this problem, though, we discovered another. One of the Copley paintings includes a `people` listing for "Mrs. Thomas Boylston (Sarah Morecock)". In this case, the gender field is empty. This is frustrating, because it does suggest that we're necessarily going to miss a number of objects that we'd otherwise include, but there isn't much we can do about it unless we investigate all the items in which `gender` is designated as `unknown`.

The last problem is most interesting, for our purposes. In the results in the video, both the second and twelfth items are collaborations between Fragonard and his sister-in-law, Marguerite Gérard. Both are recorded as artists responsible for the painting. It's this double listing that's causing the problem. This is especially interesting, because Gérard is acknowledged to have actually worked on these paintings. She wasn't just a sitter. This is exactly the kind of record that we want.

In order to solve this, we're going to have to do some digging. Let's check out the types of both all the different objects involved. Play around with this line of code to get a sense of the different objects held in your dictionary at each level. This will help us when it comes to figuring out how to solve the bug.

In [ ]:
type(extracted_records[12]['people'])

```python
extracted_records => list
extracted_records[12] => dict
extracted_records[12]['people'] => list
extracted_records[12]['people'][0] => dict
extracted_records[12]['people'][0]['gender'] => str
```

Our problem, then, is that that we want to match multiple string values that are held within the list of dictionaries called `people`. Fortunately, there's an easy way we can do that.

[This Stack Overflow page](https://stackoverflow.com/questions/8653516/python-list-of-dictionaries-search) provides a range of options for how to solve this problem, which you might find by searching Google for the keywords "```python search list of dictionaries```". To solve this problem, we'll write a search function that checks each object in `people` for its `gender`. (Stephen says this is the "fifth solution" on the Stack Overflow page, but that may well have changed since the video was made.)

The following cell solves the original problem that was causing our first script to crash, in that it only checks for `gender` if the `people` key exists. If it doesn't, it passes on to the next record. (Keep in mind that we're only printing the name of the first person listed as responsible for the work. We'll fill in the rest of the data soon.)

In [ ]:
def search(gender):
    for i in record['people']:
        if i['gender'] == gender:
            print("{}\t{}".format(record['people'][0]['name'],record['title']))

print("views\tname\t\t\ttitle")
print()
for record in extracted_records:
    if 'people' in record:
        search("male")

### Iterating to create a database of objects by women

Unsurprisingly, of the hundred records on the first page, we only find three that were produced by female artists. We're definitely going to have to loop through the whole of the Museum's eighteenth-century holdings to get anything like a satisfying dataset! 

We could set this up in a regular loop, which would query the API as fast as our processors can go, which can produce many queries per second, and is usually limited more by network speed than by processor speed. However, this can put a strain on the API endpoint, so it can be good practice to build in timers when making many requests. Sometimes an API will specify a number of requests/second that you're allowed to make, sometimes not. Putting even a fraction of a second delay in your code will help make sure that you don't accidentally get yourself banned from the API. The dictionary `time` will allow us to do that.

In [ ]:
import time

We'll set an empty dictionary called `objects by women`, and with each loop we'll add those that fit our criteria. Initially we'll set the page count as `1`, because we'll increase that number incrementally with each loop, so that we capture each page, right to the end of the collection. Finally, we'll need to slightly modify the output parameters of the function we wrote, because we want to grab the entire json object for each item made by a woman.

In [ ]:
complete_18C = ham_query("object", century="18th century", sort="totalpageviews", sortorder="desc", size=100, page=1).json()

objects_by_women = []
keepGoing = True
page = 1

def search(gender):
    for person in record['people']:
        if person['gender'] == gender:
            objects_by_women.append(record)
            break

while keepGoing:
    R = ham_query("object", century="18th century", size=100, page=page)
    time.sleep(0.5)
    records = R.json()["records"]
    for record in records:
        if 'people' in record:
            search("female")
    if complete_18C['info']['pages'] == page:
        keepGoing = False
    else:
        page += 1

If you want to check that this script has looped all the way to the end, check the `page` variable. It should correspond to the number of pages we expect, given the information we generated about the corpus earlier. Then, let's see the number of works by women we found.

In [ ]:
page

In [ ]:
len(objects_by_women)

It should be obvious at this point, but you'll probably find more objects by women than we did in the video, either because the Art Museum updated their metadata or because they acquired new works.

## Exporting Your Data

Now that we have a corpus of works, let's move it outside of this notebook. It's common to see <b>CSV</b> files used for this purpose, because they're a simple, plaintext spreadsheet file, and most programming languages and applications can parse it.  We could use the relatively low level `csv` library, but instead, let's use a higher level library, `pandas`, which we'll be using throughout the course.

In [ ]:
import pandas as pd # You'll see `pd` as a common invocation of pandas, really only because it's used so often

### First, Convert Your Data to a Dataframe

Pandas thinks of things in terms of <b>dataframes</b>, which will be familiar if you work in R. Basically, they're really efficient arrays of data. They also translate really well to a tabular format.

Since we have a list of dictionaries with consistent keys, there's a good chance this process will basically muscle our data into a tabular format, without our having to do any additional work, like naming columns and rows:

In [ ]:
df = pd.DataFrame(objects_by_women)
df

It worked! Now let's export our results to a CSV.

### Write the Final CSV File

From here, our export process is really easy. We just tell the data to become a `csv` file, and it happens. You'll be able to find the output file in the same directory to which this notebook is currently keyed. If you look at the column for `people`, you'll see that many records do contain multiple entries, some male, some female, but every object has at least one woman who participated in its creation.

In [ ]:
df.to_csv("objects_by_women.csv",index=None)

# Data collected!